# 第1步：导入库和加载图像

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Noto Sans CJK SC', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 加载图像
img = cv2.imread("小黑1.jpg")  # 替换为你的图片路径
if img is None:
    raise ValueError("无法加载图像，请检查文件路径。")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8,6),dpi=300)
plt.imshow(img_rgb)
plt.title('原始拼豆图纸')
plt.axis('off')
plt.show()

print(f"图像尺寸: {img.shape}")

# 第2步：多通道梯度计算

In [ ]:
def get_gradient(channel):
    grad_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=3)
    return np.sqrt(grad_x**2 + grad_y**2)

# RGB三通道分别计算
b, g, r = cv2.split(img)
grad_r = get_gradient(r)
grad_g = get_gradient(g)
grad_b = get_gradient(b)

# LAB色差通道
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b_lab = cv2.split(lab)
grad_a = get_gradient(a)  # 红绿色差
grad_b_lab = get_gradient(b_lab)  # 黄蓝色差

# 可视化各通道梯度
fig, axes = plt.subplots(2, 3, figsize=(15, 8), dpi=300)
axes[0,0].imshow(grad_r, cmap='hot'), axes[0,0].set_title('R通道梯度'), axes[0,0].axis('off')
axes[0,1].imshow(grad_g, cmap='hot'), axes[0,1].set_title('G通道梯度'), axes[0,1].axis('off')
axes[0,2].imshow(grad_b, cmap='hot'), axes[0,2].set_title('B通道梯度'), axes[0,2].axis('off')
axes[1,0].imshow(grad_a, cmap='hot'), axes[1,0].set_title('A通道梯度(红绿色差)'), axes[1,0].axis('off')
axes[1,1].imshow(grad_b_lab, cmap='hot'), axes[1,1].set_title('B通道梯度(黄蓝色差)'), axes[1,1].axis('off')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()

# 第3步：智能权重融合

In [ ]:
# 计算每个通道的平均强度作为权重
weights = {
    'r': np.mean(grad_r),
    'g': np.mean(grad_g),
    'b': np.mean(grad_b),
    'a': np.mean(grad_a),
    'b_lab': np.mean(grad_b_lab)
}

print("各通道平均梯度强度:")
for k, v in weights.items():
    print(f"{k}: {v:.2f}")

# 归一化并融合
total_weight = sum(weights.values())
fused_gradient = (
    (weights['r']/total_weight) * grad_r +
    (weights['g']/total_weight) * grad_g +
    (weights['b']/total_weight) * grad_b +
    (weights['a']/total_weight) * grad_a * 1.5 +    # 色差通道加权
    (weights['b_lab']/total_weight) * grad_b_lab * 1.5
)

# 使用60%阈值的二值图进行线检测
threshold_60 = fused_gradient > np.percentile(fused_gradient, 55)
binary_60 = (threshold_60 * 255).astype(np.uint8)

plt.figure(figsize=(12, 4),dpi=300)
plt.subplot(131), plt.imshow(grad_r, cmap='hot'), plt.title('单通道(R)'), plt.axis('off')
plt.subplot(132), plt.imshow(fused_gradient, cmap='hot'), plt.title('多通道融合'), plt.axis('off')
plt.subplot(133), plt.imshow(binary_60, cmap='gray'), plt.title('60%阈值预览'), plt.axis('off')
plt.show()

# 第4步：基于形态学的网格线和交叉点检测（最终版本）

In [ ]:
def extract_grid_lines_morphology(binary_img, line_length=15, iterations=2):
    """
    使用形态学操作提取水平线和垂直线
    """
    print(f"输入图像尺寸: {binary_img.shape}")

    # 创建结构元素
    # 水平线检测核 - 长水平线，窄垂直线
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (line_length, 1))
    # 垂直线检测核 - 窄水平线，长垂直线
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, line_length))

    # 形态学开运算提取线条
    horizontal_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                      horizontal_kernel, iterations=iterations)
    vertical_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                    vertical_kernel, iterations=iterations)

    print(f"提取的水平线像素数: {np.sum(horizontal_lines > 0)}")
    print(f"提取的垂直线像素数: {np.sum(vertical_lines > 0)}")

    return horizontal_lines, vertical_lines

def find_line_positions(line_mask, direction='horizontal'):
    """
    从线条掩码中提取线条的精确位置
    """
    if direction == 'horizontal':
        # 水平线：在y方向上投影
        projection = np.sum(line_mask, axis=1)
    else:
        # 垂直线：在x方向上投影
        projection = np.sum(line_mask, axis=0)

    # 找到投影的峰值位置
    from scipy.signal import find_peaks

    # 设置峰值检测参数
    height_threshold = np.max(projection) * 0.1  # 10%的最大值作为阈值
    min_distance = 5  # 最小间距

    peaks, _ = find_peaks(projection, height=height_threshold, distance=min_distance)

    return peaks, projection

def detect_grid_intersections_morphology(horizontal_lines, vertical_lines):
    """
    检测网格交叉点
    """
    # 找到交叉点：水平线和垂直线的交集
    intersections_mask = cv2.bitwise_and(horizontal_lines, vertical_lines)

    # 使用连通组件找到交叉点位置
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        intersections_mask, connectivity=8)

    # 过滤掉面积太小的组件
    min_area = 2  # 最小面积阈值
    valid_intersections = []

    for i in range(1, num_labels):  # 跳过背景标签0
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            cx, cy = centroids[i]
            valid_intersections.append((cx, cy))

    print(f"检测到 {len(valid_intersections)} 个网格交叉点")

    return np.array(valid_intersections), intersections_mask

# 执行形态学网格检测
print("=== 开始形态学网格检测 ===")

# 1. 提取水平线和垂直线
horizontal_lines, vertical_lines = extract_grid_lines_morphology(
    binary_60, line_length=60, iterations=2)

# 2. 获取线条位置
horizontal_positions, h_projection = find_line_positions(horizontal_lines, 'horizontal')
vertical_positions, v_projection = find_line_positions(vertical_lines, 'vertical')

print(f"检测到 {len(horizontal_positions)} 条水平线")
print(f"检测到 {len(vertical_positions)} 条垂直线")

# 3. 检测交叉点
grid_intersections, intersection_mask = detect_grid_intersections_morphology(
    horizontal_lines, vertical_lines)

# 第5步：形态学检测结果可视化

In [ ]:
plt.figure(figsize=(20, 16), dpi=300)

# 1. 原始图像
plt.subplot(3, 4, 1)
plt.imshow(img_rgb)
plt.title('原始拼豆图纸')
plt.axis('off')

# 2. 60%阈值二值图
plt.subplot(3, 4, 2)
plt.imshow(binary_60, cmap='gray')
plt.title('60%阈值二值图')
plt.axis('off')

# 3. 提取的水平线
plt.subplot(3, 4, 3)
plt.imshow(horizontal_lines, cmap='gray')
plt.title(f'水平线提取\n{len(horizontal_positions)}条线')
plt.axis('off')

# 4. 提取的垂直线
plt.subplot(3, 4, 4)
plt.imshow(vertical_lines, cmap='gray')
plt.title(f'垂直线提取\n{len(vertical_positions)}条线')
plt.axis('off')

# 5. 水平线投影分析
plt.subplot(3, 4, 5)
plt.plot(h_projection, range(len(h_projection)), color='blue')
plt.scatter(h_projection[horizontal_positions], horizontal_positions,
           color='red', s=50, zorder=5)
plt.title('水平线投影')
plt.xlabel('投影强度')
plt.ylabel('Y坐标')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)

# 6. 垂直线投影分析
plt.subplot(3, 4, 6)
plt.plot(v_projection, color='green')
plt.scatter(vertical_positions, v_projection[vertical_positions],
           color='red', s=50, zorder=5)
plt.title('垂直线投影')
plt.xlabel('X坐标')
plt.ylabel('投影强度')
plt.grid(True, alpha=0.3)

# 7. 交叉点掩码
plt.subplot(3, 4, 7)
plt.imshow(intersection_mask, cmap='gray')
plt.title('交叉点掩码')
plt.axis('off')

# 8. 合并的线条图
plt.subplot(3, 4, 8)
combined_lines = cv2.bitwise_or(horizontal_lines, vertical_lines)
plt.imshow(combined_lines, cmap='gray')
plt.title('合并线条')
plt.axis('off')

# 9. 线条叠加在原图上
plt.subplot(3, 4, 9)
lines_overlay = img_rgb.copy()
# 水平线 - 绿色
lines_overlay[horizontal_lines > 0] = [0, 255, 0]
# 垂直线 - 红色
lines_overlay[vertical_lines > 0] = [255, 0, 0]
# 交叉点 - 黄色
lines_overlay[intersection_mask > 0] = [255, 255, 0]
plt.imshow(lines_overlay)
plt.title('线条叠加原图\n红=垂直,绿=水平,黄=交叉')
plt.axis('off')

# 10. 交叉点标记
plt.subplot(3, 4, 10)
intersection_result = img_rgb.copy()
for i, (x, y) in enumerate(grid_intersections):
    cv2.circle(intersection_result, (int(x), int(y)), 4, (255, 0, 0), -1)
    # 每50个点标注序号
    if i % 50 == 0:
        cv2.putText(intersection_result, str(i), (int(x)+5, int(y)-5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
plt.imshow(intersection_result)
plt.title(f'网格交叉点\n共{len(grid_intersections)}个')
plt.axis('off')

# 11. 网格间距分析
plt.subplot(3, 4, 11)
if len(grid_intersections) > 4:
    from scipy.spatial.distance import cdist

    # 计算最近邻距离
    distances = cdist(grid_intersections, grid_intersections)
    np.fill_diagonal(distances, np.inf)
    min_distances = np.min(distances, axis=1)

    plt.hist(min_distances, bins=20, alpha=0.7, color='blue', edgecolor='black')

    # 统计信息
    median_spacing = np.median(min_distances)
    mean_spacing = np.mean(min_distances)
    std_spacing = np.std(min_distances)

    plt.axvline(median_spacing, color='red', linestyle='--',
               label=f'中位数: {median_spacing:.1f}')
    plt.axvline(mean_spacing, color='green', linestyle='--',
               label=f'平均值: {mean_spacing:.1f}')

    plt.title(f'网格间距分布\n标准差: {std_spacing:.1f}')
    plt.xlabel('距离(像素)')
    plt.ylabel('频次')
    plt.legend()
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, '交点数据不足', ha='center', va='center',
             transform=plt.gca().transAxes)
    plt.title('网格间距分析')

# 12. 交点分布图
plt.subplot(3, 4, 12)
if len(grid_intersections) > 0:
    plt.scatter(grid_intersections[:, 0], grid_intersections[:, 1],
               alpha=0.7, s=20, c='red')
    plt.title(f'交点分布图\n{len(grid_intersections)}个交点')
    plt.xlabel('X坐标')
    plt.ylabel('Y坐标')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)

    # 显示网格的大概范围
    x_min, x_max = np.min(grid_intersections[:, 0]), np.max(grid_intersections[:, 0])
    y_min, y_max = np.min(grid_intersections[:, 1]), np.max(grid_intersections[:, 1])

    plt.text(0.02, 0.98, f'X范围: {x_min:.0f}-{x_max:.0f}\nY范围: {y_min:.0f}-{y_max:.0f}',
             transform=plt.gca().transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

In [ ]:
# 第4步：基于形态学的网格线和交叉点检测（自适应优化版本）
def extract_grid_lines_morphology_adaptive(binary_img, min_line_length=5, max_line_length=30, iterations=2):
    """
    自适应形态学操作提取水平线和垂直线
    """
    print(f"输入图像尺寸: {binary_img.shape}")

    # 初步检测以估算网格密度
    initial_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 1))
    temp_horizontal = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, initial_kernel, iterations=1)
    temp_projection = np.sum(temp_horizontal, axis=1)

    from scipy.signal import find_peaks
    temp_peaks, _ = find_peaks(temp_projection, height=np.max(temp_projection) * 0.1, distance=5)

    # 计算网格密度并自适应调整线段长度
    density = len(temp_peaks) / binary_img.shape[0] if len(temp_peaks) > 0 else 0.01
    adaptive_line_length = max(min_line_length, min(max_line_length, int(0.15/density * 100)))

    print(f"检测到网格密度: {density:.4f} 线/像素")
    print(f"自适应线段长度: {adaptive_line_length} 像素")

    # 使用自适应长度创建结构元素
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (adaptive_line_length, 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, adaptive_line_length))

    # 形态学开运算提取线条
    horizontal_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                      horizontal_kernel, iterations=iterations)
    vertical_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                    vertical_kernel, iterations=iterations)

    print(f"提取的水平线像素数: {np.sum(horizontal_lines > 0)}")
    print(f"提取的垂直线像素数: {np.sum(vertical_lines > 0)}")

    return horizontal_lines, vertical_lines, adaptive_line_length

def detect_grid_intersections_refined(horizontal_lines, vertical_lines, img_shape):
    """
    优化的网格交叉点检测（使用K-means聚类精确定位）
    """
    # 找到交叉点：水平线和垂直线的交集
    intersections_mask = cv2.bitwise_and(horizontal_lines, vertical_lines)

    # 获取所有交叉点像素坐标
    intersection_pixels = np.argwhere(intersections_mask > 0)

    if len(intersection_pixels) == 0:
        print("未检测到交叉点")
        return np.array([]), intersections_mask

    # 计算聚类粒度（基于图像尺寸自适应）
    cluster_granularity = max(20, min(100, int(max(img_shape) / 50)))
    expected_clusters = max(1, len(intersection_pixels) // cluster_granularity)

    print(f"交叉点像素总数: {len(intersection_pixels)}")
    print(f"聚类粒度: {cluster_granularity}, 期望聚类数: {expected_clusters}")

    if expected_clusters >= len(intersection_pixels):
        # 如果期望聚类数太大，直接使用连通组件方法
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
            intersections_mask, connectivity=8)

        valid_intersections = []
        min_area = 2

        for i in range(1, num_labels):
            area = stats[i, cv2.CC_STAT_AREA]
            if area >= min_area:
                cx, cy = centroids[i]
                valid_intersections.append((cx, cy))

        refined_intersections = np.array(valid_intersections)
        print(f"连通组件方法检测到 {len(refined_intersections)} 个交叉点")

    else:
        # 使用K-means聚类优化交叉点位置
        coords = np.float32(intersection_pixels[:, [1, 0]])  # 转换为(x,y)格式

        # K-means参数设置
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
        attempts = 10
        flags = cv2.KMEANS_RANDOM_CENTERS

        # 执行K-means聚类
        _, labels, centers = cv2.kmeans(coords, expected_clusters, None, criteria, attempts, flags)

        # 过滤聚类结果：去除边界附近的异常点
        margin = 10  # 边界margin
        h, w = img_shape[:2]
        valid_centers = []

        for center in centers:
            x, y = center
            if margin < x < w - margin and margin < y < h - margin:
                valid_centers.append((x, y))

        refined_intersections = np.array(valid_centers)
        print(f"K-means聚类方法检测到 {len(refined_intersections)} 个交叉点")

    return refined_intersections, intersections_mask

# 执行自适应形态学网格检测
print("=== 开始自适应形态学网格检测 ===")

# 1. 自适应提取水平线和垂直线
horizontal_lines, vertical_lines, used_line_length = extract_grid_lines_morphology_adaptive(
    binary_60, min_line_length=5, max_line_length=1000, iterations=2)

# 2. 获取线条位置
horizontal_positions, h_projection = find_line_positions(horizontal_lines, 'horizontal')
vertical_positions, v_projection = find_line_positions(vertical_lines, 'vertical')

print(f"检测到 {len(horizontal_positions)} 条水平线")
print(f"检测到 {len(vertical_positions)} 条垂直线")

# 3. 优化的交叉点检测
grid_intersections, intersection_mask = detect_grid_intersections_refined(
    horizontal_lines, vertical_lines, binary_60.shape)

# 4. 验证检测质量
if len(grid_intersections) > 4:
    from scipy.spatial.distance import cdist
    distances = cdist(grid_intersections, grid_intersections)
    np.fill_diagonal(distances, np.inf)
    min_distances = np.min(distances, axis=1)

    spacing_consistency = 1 - (np.std(min_distances) / np.mean(min_distances))
    print(f"网格间距一致性: {spacing_consistency:.3f} (越接近1越规整)")
    print(f"平均网格间距: {np.mean(min_distances):.1f} ± {np.std(min_distances):.1f} 像素")

In [ ]:
# 第5步：自适应检测结果可视化（增强版）
plt.figure(figsize=(24, 16), dpi=300)

# 1. 原始图像
plt.subplot(3, 5, 1)
plt.imshow(img_rgb)
plt.title('原始拼豆图纸')
plt.axis('off')

# 2. 自适应二值图
plt.subplot(3, 5, 2)
plt.imshow(binary_60, cmap='gray')
plt.title(f'阈值二值图\n自适应线长:{used_line_length}px')
plt.axis('off')

# 3. 自适应水平线提取
plt.subplot(3, 5, 3)
plt.imshow(horizontal_lines, cmap='gray')
plt.title(f'自适应水平线\n{len(horizontal_positions)}条线')
plt.axis('off')

# 4. 自适应垂直线提取
plt.subplot(3, 5, 4)
plt.imshow(vertical_lines, cmap='gray')
plt.title(f'自适应垂直线\n{len(vertical_positions)}条线')
plt.axis('off')

# 5. 网格密度分析
plt.subplot(3, 5, 5)
if len(horizontal_positions) > 1 and len(vertical_positions) > 1:
    h_spacings = np.diff(np.sort(horizontal_positions))
    v_spacings = np.diff(np.sort(vertical_positions))

    plt.hist(h_spacings, alpha=0.7, label=f'水平间距\n(CV:{np.std(h_spacings)/np.mean(h_spacings):.3f})', bins=10)
    plt.hist(v_spacings, alpha=0.7, label=f'垂直间距\n(CV:{np.std(v_spacings)/np.mean(v_spacings):.3f})', bins=10)
    plt.title('线段间距分布')
    plt.xlabel('间距(像素)')
    plt.ylabel('频次')
    plt.legend()
    plt.grid(True, alpha=0.3)

# 6. 投影分析对比
plt.subplot(3, 5, 6)
plt.plot(h_projection, range(len(h_projection)), color='blue', alpha=0.7)
plt.scatter(h_projection[horizontal_positions], horizontal_positions,
           color='red', s=30, zorder=5)
plt.title(f'水平线投影\n峰值数:{len(horizontal_positions)}')
plt.xlabel('投影强度')
plt.ylabel('Y坐标')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)

plt.subplot(3, 5, 7)
plt.plot(v_projection, color='green', alpha=0.7)
plt.scatter(vertical_positions, v_projection[vertical_positions],
           color='red', s=30, zorder=5)
plt.title(f'垂直线投影\n峰值数:{len(vertical_positions)}')
plt.xlabel('X坐标')
plt.ylabel('投影强度')
plt.grid(True, alpha=0.3)

# 8. 交叉点掩码
plt.subplot(3, 5, 8)
plt.imshow(intersection_mask, cmap='gray')
plt.title('优化交叉点掩码')
plt.axis('off')

# 9. 线条合成图
plt.subplot(3, 5, 9)
combined_lines = cv2.bitwise_or(horizontal_lines, vertical_lines)
plt.imshow(combined_lines, cmap='gray')
plt.title('合成网格线条')
plt.axis('off')

# 10. 彩色叠加效果
plt.subplot(3, 5, 10)
adaptive_overlay = img_rgb.copy()
# 水平线 - 绿色
adaptive_overlay[horizontal_lines > 0] = [0, 255, 0]
# 垂直线 - 红色
adaptive_overlay[vertical_lines > 0] = [255, 0, 0]
# 交叉点 - 黄色
adaptive_overlay[intersection_mask > 0] = [255, 255, 0]
plt.imshow(adaptive_overlay)
plt.title(f'自适应检测结果\n红=垂直,绿=水平,黄=交叉')
plt.axis('off')

# 11. 精确交叉点标记
plt.subplot(3, 5, 11)
refined_result = img_rgb.copy()
for i, (x, y) in enumerate(grid_intersections):
    cv2.circle(refined_result, (int(x), int(y)), 3, (255, 0, 0), -1)
    # 每30个点标注序号
    if i % 30 == 0:
        cv2.putText(refined_result, str(i), (int(x)+5, int(y)-5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
plt.imshow(refined_result)
plt.title(f'精确交叉点\n共{len(grid_intersections)}个')
plt.axis('off')

# 12. 网格质量评估
plt.subplot(3, 5, 12)
if len(grid_intersections) > 4:
    from scipy.spatial.distance import cdist
    distances = cdist(grid_intersections, grid_intersections)
    np.fill_diagonal(distances, np.inf)
    min_distances = np.min(distances, axis=1)

    plt.hist(min_distances, bins=15, alpha=0.7, color='purple', edgecolor='black')

    median_spacing = np.median(min_distances)
    mean_spacing = np.mean(min_distances)
    std_spacing = np.std(min_distances)

    plt.axvline(median_spacing, color='red', linestyle='--', label=f'中位数: {median_spacing:.1f}')
    plt.axvline(mean_spacing, color='orange', linestyle='--', label=f'均值: {mean_spacing:.1f}')

    consistency = 1 - (std_spacing / mean_spacing)
    plt.title(f'网格间距质量\n一致性: {consistency:.3f}')
    plt.xlabel('距离(像素)')
    plt.ylabel('频次')
    plt.legend(fontsize=8)
    plt.grid(True, alpha=0.3)

# 13. 交点分布热图
plt.subplot(3, 5, 13)
if len(grid_intersections) > 0:
    plt.scatter(grid_intersections[:, 0], grid_intersections[:, 1],
               alpha=0.6, s=15, c=range(len(grid_intersections)), cmap='viridis')
    plt.colorbar(label='交点序号', shrink=0.8)
    plt.title(f'交点空间分布\n{len(grid_intersections)}个点')
    plt.xlabel('X坐标')
    plt.ylabel('Y坐标')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)

# 14. 自适应参数展示
plt.subplot(3, 5, 14)
params_text = f"""自适应参数总结:

线段长度: {used_line_length} px
水平线数: {len(horizontal_positions)}
垂直线数: {len(vertical_positions)}
交叉点数: {len(grid_intersections)}

理论网格: {len(horizontal_positions)}×{len(vertical_positions)}
理论交点: {len(horizontal_positions) * len(vertical_positions)}
检测效率: {len(grid_intersections)/(len(horizontal_positions) * len(vertical_positions))*100 if len(horizontal_positions) * len(vertical_positions) > 0 else 0:.1f}%

图像尺寸: {binary_60.shape[1]}×{binary_60.shape[0]}
"""

plt.text(0.05, 0.95, params_text, transform=plt.gca().transAxes,
         fontsize=9, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.7))
plt.title('检测参数统计')
plt.axis('off')

# 15. 局部放大效果
plt.subplot(3, 5, 15)
if len(grid_intersections) > 0:
    # 选择中心区域放大
    h, w = img_rgb.shape[:2]
    center_x, center_y = w//2, h//2
    crop_size = min(200, h//4, w//4)

    crop_img = img_rgb[center_y-crop_size//2:center_y+crop_size//2,
                       center_x-crop_size//2:center_x+crop_size//2].copy()

    # 在裁剪区域标记交点
    local_intersections = 0
    for x, y in grid_intersections:
        if (center_x-crop_size//2 <= x <= center_x+crop_size//2 and
            center_y-crop_size//2 <= y <= center_y+crop_size//2):
            x_local = int(x - (center_x-crop_size//2))
            y_local = int(y - (center_y-crop_size//2))
            cv2.circle(crop_img, (x_local, y_local), 2, (255, 0, 0), -1)
            local_intersections += 1

    plt.imshow(crop_img)
    plt.title(f'局部放大\n{local_intersections}个交点')
    plt.axis('off')

plt.tight_layout()
plt.show()